## Hyperparameter search for StatePredictor on NACA0012 data
Do this tutorial after [`run.ipynb`](./run.ipynb).

You might be wondering how we arrived at the input configurations in the 'Optimizing' and the 'Improving run time' sections in `run.ipynb`. The method `deepk.hyp_search:run_hyp_search()` can perform hyperparameter search on either `StatePredictor` or `TrajectoryPredictor`. This sweeps the values of the different inputs to the class and its methods. Each configuration is trained and the loss and ANAE statistics across epochs recorded for training data (and validation data, if provided). These results can then be used to select 'good' input configurations.

In [1]:
from deepk.hyp_search import run_hyp_search
from deepk.state_predictor import StatePredictor_DataHandler
from deepk import utils

### Load data

In [2]:
import pickle
with open('./data.pkl', 'rb') as f:
    data = pickle.load(f)

The type of Data Handler we create determines the predictor model on which hyperparameter search is performed. In this case, we create a `StatePredictor_DataHandler`. This automatically sets the hyperparameter search to run on `StatePredictor` models.

In [3]:
dh = StatePredictor_DataHandler(
    Xtr=data['Xtr'], ttr=data['ttr'],
    Xva=data['Xva'], tva=data['tva']
)

Note that test data is not used in hyperparameter search. It is recommended to provide validaion data, so that validation loss and ANAE statistics can also be collected.

### Options for hyperparameter search
The `StatePredictor` class has possible inputs:
- `dh`
- `rank`
- `encoded_size`
- `encoder_hidden_layers`
- `decoder_hidden_layers`
- `batch_norm`

Its `train_net()` method has possible inputs:
- `numepochs`
- `early_stopping`
- `early_stopping_metric`
- `lr`
- `weight_decay`
- `decoder_loss_weight`
- `Kreg`
- `cond_threshold`
- `clip_grad_norm`
- `clip_grad_value`

All of these inputs, with the exception of `dh` already defined above, can be swept together in the hyperparameter search.

For more on the inputs, see the [documentation of `StatePredictor`](htps://TODO).

Let us define some values to sweep over.

In [4]:
hyp_options = {
    'encoded_size': 50, # 1 option
    'encoder_hidden_layers': [ [100,100], [200,100,50] ], # 2 options
    'numepochs': [100, 200, 300], # 3 options
    'weight_decay': 1e-5 # 1 option
}

Options that are not provided will revert to defaults.

### Run hyperparameter search
Let us now run the hyperparameter search.

In [5]:
try:
    run_hyp_search(
        dh = dh,
        hyp_options = hyp_options
    )
except ValueError as e:
    print(e)

'rank' is a required argument to StatePredictor, so 'hyp_options' must include a key-value pair for {'rank': <rank_values>}


Uh oh, you got a `ValueError: 'rank' is a required argument to StatePredictor, so 'hyp_options' must include a key-value pair for {'rank': <rank_values>}`. As it says, we need to specify one or more options for `rank` since this is a required argument that does not have a default value. Let us specify:

In [6]:
hyp_options['rank'] = [4, 6]

`hyp_options` now has a total of 12 options = 2 for `'rank'` times 2 for `'encoder_hidden_layers'` times 3 for `'numepochs'`. Let us run it again.

In [7]:
output_folder = run_hyp_search(
    dh = dh,
    hyp_options = hyp_options
)

  0%|          | 0/12 [00:00<?, ?it/s]


********************************************************************************
Starting StatePredictor hyperparameter search. Results will be stored in /Users/sourya/work/Essence/deep-koopman/examples/naca0012/hyp_search_6uznr5nGFUAnAFSisJZaEP/hyp_search_results.csv.

Performing total 12 runs. You can interrupt the script at any time (e.g. Ctrl+C), and intermediate results will be available in the above file.

Log of the entire hyperpaameter search, as well as logs of failed StatePredictor runs will also be stored in the same folder.

Hyperparameters' sweep ranges:
rank : 4, 6
encoded_size : 50
encoder_hidden_layers : [100, 100], [200, 100, 50]
numepochs : 100, 200, 300
weight_decay : 1e-05
********************************************************************************



100%|██████████| 12/12 [00:34<00:00,  2.83s/it]


### Results
Hooray, it now runs successfully and returns a path to a folder. This contains a file `hyp_search_results.csv`. Let us open it.

In [8]:
import pandas as pd
import os
df = pd.read_csv(os.path.join(output_folder, 'hyp_search_results.csv'))
df

,UUID,rank,encoder_hidden_layers,numepochs,avg_recon_loss_tr,final_recon_loss_tr,avg_recon_loss_va,best_recon_loss_va,bestep_recon_loss_va,avg_lin_loss_tr,...,avg_lin_anae_tr,final_lin_anae_tr,avg_lin_anae_va,best_lin_anae_va,bestep_lin_anae_va,avg_pred_anae_tr,final_pred_anae_tr,avg_pred_anae_va,best_pred_anae_va,bestep_pred_anae_va
0,RjoHq8UdW3nrGLVM7ypCwT,4,"[100, 100]",200,0.003137,0.000189,0.016616,0.003628,195,0.000171,...,5.749075,0.447996,12.580232,1.595680,61,170.359142,41.137810,369.384500,133.523148,200
1,2i5bqBnxRqbbBTQxTHv52L,4,"[100, 100]",300,0.002747,0.000190,0.016442,0.003932,299,0.000140,...,6.168933,0.929518,32.996355,2.990858,35,129.926650,32.684006,188.648688,64.504990,297
2,LEDBapbdDM3tEYbK2ewKgx,6,"[100, 100]",300,0.002153,0.000160,0.012588,0.003156,300,0.000141,...,19.242707,1.750130,66.426500,4.598801,35,104.343127,28.304497,203.419064,50.927658,298
3,UWFcU3QYqPW3M5DtEWUnRX,4,"[200, 100, 50]",300,0.002066,0.000167,0.013450,0.003439,297,0.000246,...,4.595422,0.188614,18.033944,1.376639,75,73.100246,24.763781,125.523613,53.717072,294
4,KzEqFfihqt86dAN2itm8sT,6,"[100, 100]",200,0.003298,0.000182,0.017321,0.003852,200,0.000223,...,7.635712,0.833945,26.515012,3.888892,61,164.118183,37.730003,261.344573,72.907814,200
5,GqyNSPjMUTiV65wRe3BMb3,4,"[100, 100]",100,0.006101,0.001256,0.029609,0.010294,100,0.000137,...,9.127914,0.879933,40.536236,4.116151,32,268.769232,156.522430,412.923551,214.694092,55
6,Fm8EnzZGQHxZfT3i6UK9L5,6,"[200, 100, 50]",300,0.001747,0.000167,0.011536,0.003469,291,0.000390,...,6.870667,0.642977,24.568308,11.992144,70,76.572726,24.552498,129.200998,48.219223,182
7,4F6aSdBKyJnEtkrPM32iCE,6,"[200, 100, 50]",200,0.002881,0.000268,0.017636,0.004320,145,0.000392,...,6.536044,0.933677,18.579365,1.481332,144,106.675288,31.455770,162.725131,62.791615,196
8,nKEB2xYLG2wUhFi4qZ2hPu,6,"[100, 100]",100,0.005863,0.000846,0.027333,0.008082,100,0.000441,...,11.222060,0.678201,26.312276,2.411977,65,246.105336,125.951302,393.237248,211.418961,100
9,AoW6zDBkTBAMMEQYa6CbJk,4,"[200, 100, 50]",200,0.003247,0.000206,0.019985,0.005011,183,0.000704,...,8.867508,0.360216,18.778859,1.950196,82,110.610109,27.640871,155.047949,55.653294,193


This contains loss and ANAE statistics for all 12 runs, as `rank`, `encoder_hidden_layers`, and `numepochs` are swept. The statistics collected for each performance metric `perf` are:
- `avg_perf_tr` - Average of metric for training data over all epochs.
- `final_perf_tr` - Value of metric in last epoch of training data.

If validation data is provided:
- `avg_perf_va` - Average of metric for validation data over all epochs.
- `best_perf_va` - Best value of metric for validation data over all epochs.
- `bestep_perf_va` - Epoch at which best value of metric for validation data was obtained.

The 12 results from top to bottom are arranged from best to worst of the `sort_key` of `run_hyp_search()`, which is by default set to `'avg_total_loss_va'`. This is because total loss on validation data averaged across all epochs is an important metric for quantifying performance. Other important ones are `avg_pred_anae_va` and `avg_pred_loss_va`.

Let's view this.

In [9]:
df_truncated = df[['rank', 'encoder_hidden_layers', 'numepochs', 'avg_total_loss_va']]
df_truncated

,rank,encoder_hidden_layers,numepochs,avg_total_loss_va
0,4,"[100, 100]",200,0.000513
1,4,"[100, 100]",300,0.000520
2,6,"[100, 100]",300,0.000572
3,4,"[200, 100, 50]",300,0.000607
4,6,"[100, 100]",200,0.000647
5,4,"[100, 100]",100,0.000730
6,6,"[200, 100, 50]",300,0.000765
7,6,"[200, 100, 50]",200,0.000829
8,6,"[100, 100]",100,0.000988
9,4,"[200, 100, 50]",200,0.001197


You can see from these results that `encoder_hidden_layers = [100, 100]` is clewrly doing better than `[200, 100, 50]`. Also, training for more epochs is generally better. These insights are very helpful in selecting a good combination of hyperparameters.

### Ignoring first few epochs
Look back at the results in [`run.ipynb`](./run.ipynb). Notice how the performance is erratic for the first few epochs. As an example:
<img src="./skewed_initial_epochs_example.png" width="300"/>

A few erratic initial epochs can skew the average statistics significantly. This is why the `run_hyp_search()` method has an argument `avg_ignore_initial_epochs`, which specifies the number of initial epochs to ignore for average calculations. Let us set this to `100`, remove the 100 epoch option, and run again.

In [10]:
hyp_options['numepochs'].remove(100)
output_folder = run_hyp_search(
    dh = dh,
    hyp_options = hyp_options,
    avg_ignore_initial_epochs = 100
)

df = pd.read_csv(os.path.join(output_folder, 'hyp_search_results.csv'))
df_truncated = df[['rank', 'encoder_hidden_layers', 'numepochs', 'avg_total_loss_va']]
df_truncated

  0%|          | 0/8 [00:00<?, ?it/s]


********************************************************************************
Starting StatePredictor hyperparameter search. Results will be stored in /Users/sourya/work/Essence/deep-koopman/examples/naca0012/hyp_search_7ZRpcFreheiqz5uCNNm3s9/hyp_search_results.csv.

Performing total 8 runs. You can interrupt the script at any time (e.g. Ctrl+C), and intermediate results will be available in the above file.

Log of the entire hyperpaameter search, as well as logs of failed StatePredictor runs will also be stored in the same folder.

Hyperparameters' sweep ranges:
rank : 4, 6
encoded_size : 50
encoder_hidden_layers : [100, 100], [200, 100, 50]
numepochs : 200, 300
weight_decay : 1e-05
********************************************************************************



100%|██████████| 8/8 [00:26<00:00,  3.36s/it]


,rank,encoder_hidden_layers,numepochs,avg_total_loss_va
0,4,"[200, 100, 50]",200,0.000102
1,4,"[100, 100]",200,0.000119
2,6,"[100, 100]",200,0.000121
3,6,"[100, 100]",300,0.000175
4,6,"[200, 100, 50]",200,0.000196
5,6,"[200, 100, 50]",300,0.000217
6,4,"[200, 100, 50]",300,0.000222
7,4,"[100, 100]",300,0.000334


The results look better now, and are less sensitive to outlier epochs.

### Controlling the number of runs
If you don't want to wait to run every possible configuration (which can exponentially explode as the number of options increase), you can control the number of runs using the `numruns` argument of `run_hyp_search()`. Setting this to less than the cardinality of the Cartesian product of all the values in `hyp_options` will randomly sample `numruns` runs.

As an example, let's try sampling 5 runs.

In [11]:
output_folder = run_hyp_search(
    dh = dh,
    hyp_options = hyp_options,
    avg_ignore_initial_epochs = 100,
    numruns = 5
)

df = pd.read_csv(os.path.join(output_folder, 'hyp_search_results.csv'))
df_truncated = df[['rank', 'encoder_hidden_layers', 'numepochs', 'avg_total_loss_va']]
df_truncated

  0%|          | 0/5 [00:00<?, ?it/s]


********************************************************************************
Starting StatePredictor hyperparameter search. Results will be stored in /Users/sourya/work/Essence/deep-koopman/examples/naca0012/hyp_search_92e7KQjuvk26VyqHGpZG3v/hyp_search_results.csv.

Performing total 5 runs. You can interrupt the script at any time (e.g. Ctrl+C), and intermediate results will be available in the above file.

Log of the entire hyperpaameter search, as well as logs of failed StatePredictor runs will also be stored in the same folder.

Hyperparameters' sweep ranges:
rank : 4, 6
encoded_size : 50
encoder_hidden_layers : [100, 100], [200, 100, 50]
numepochs : 200, 300
weight_decay : 1e-05
********************************************************************************



100%|██████████| 5/5 [00:15<00:00,  3.12s/it]


,rank,encoder_hidden_layers,numepochs,avg_total_loss_va
0,6,"[200, 100, 50]",200,0.000097
1,6,"[100, 100]",200,0.000118
2,4,"[200, 100, 50]",300,0.000140
3,4,"[200, 100, 50]",200,0.000141
4,4,"[100, 100]",200,0.000209


### Concluding thoughts
We highly recommend performing hyperparameter search for any problem as it can lead to massively improved results. If required, you can perform several hundred or even several thousand runs, which can take several hours to run, but the results are usually worth it.

Here's an example of an extensive hyperparameter search:
```python
output_folder = run_hyp_search(
    dh = dh,
    hyp_options = {
        'rank': [3,6,8,10,20], #5 options
        'num_encoded_states': [50,100,200,500,1000], #5 options
        'encoder_hidden_layers': [
            [100,100],[200,200],[500,500],
            [50,100],[100,50],[100,200],[200,100],[200,500],[500,200],[500,1000],[1000,500],
            [100,100,100],[200,200,200],[500,500,500],
            [50,100,200],[200,100,50],[100,200,500],[500,200,100],[200,500,1000],[1000,500,200]
        ], #20 options
        'weight_decay': [0.,1e-6,1e-5,1e-4], #4 options
        'decoder_loss_weight': [1e-3,1e-2,1e-1,1.], #4 options
        'Kreg': [0.,1e-3,1e-2], #3 options
        'clip_grad_norm': [None,5.,10.], #3 options
        'clip_grad_value': [None,2.], #2 options
    }, # total = 144,000 options
    avg_ignore_initial_epochs = 100,
    numruns = 3000 # randomly sample ~2% of the entire space
)
```